<h1> Layer.ai Air Quality Prediction Challenge 1st Place Solution</center></h1>

**TABLE OF CONTENTS**

<ul class="list-group" style="list-style-type:none;">
    <li>1. Setup </li>
    <li>2. Load Data </li>
    <li>3. Data Cleaning </li>
    <li>4. Feature Engineering </li>
    <li>5. Preprocessing </li>
    <li>6. Modeling </li>
    <ul class="list-group2" style="list-style-type:none;">
        <li>A. Define Model </li>
        <li>B. Tune Hyperparameters </li>
        <li>C. Model Training </li>
        <li>D. Model Evaluation </li>
        <li>E. Error Analysis </li>
        <li>F. Inference </li>
    </ul>
    <li>7. Submission </li>
</ul>


Links to datasets used:
1. Imputed train set can be found [here](https://app.layer.ai/nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/imputed_train)
2. Imputed test set can be found [here](https://app.layer.ai/nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/imputed_test)
3. Pseudo labels can be find [here](https://app.layer.ai/nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/pseudo_labels?v=1.1)

## Setup

In [ ]:
# Install the latest version of layer
!pip install layer --upgrade

In [ ]:
!pip install pytorch_forecasting --upgrade

In [ ]:
# !pip install pytorch-forecasting==0.9.0 && pip install pytorch-lightning==1.5.0

In [ ]:
import layer
from layer.decorators import dataset, model, resources

# Inorder to use layer please provide your own API KEY
# Get your API KEY here:
# https://app.layer.ai/me/settings/developer
API_KEY = "ENTER-API-KEY"
layer.login_with_api_key(API_KEY)

In [ ]:
# Initialise project
layer.init("zindi-air-quality-prediction-challenge")

In [ ]:
# Import core packages
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import pickle

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import random
import seaborn as sns
import sys

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import StandardScaler

import torch
from tqdm import tqdm

import warnings

# pd.set_option('display.max_rows', 1000)           
warnings.filterwarnings('ignore')

In [ ]:
def seed_everything(seed, pytorch_init=True):
    """
    Seeds basic parameters for reproducibility of results
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    if pytorch_init is True:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
SEED = 2047
seed_everything(SEED)

## Load Data

In [ ]:
train = layer.get_dataset("nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/imputed_train:1.1").to_pandas()
test = layer.get_dataset("nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/imputed_test:1.1").to_pandas()

In [ ]:
# # train = layer.get_dataset("nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/competition-data-train:1.1").to_pandas()
# # test = layer.get_dataset("nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/competition-data-test:1.1").to_pandas()
sample_submission = layer.get_dataset("nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/competition-data-samplesubmission:1.1").to_pandas()

In [ ]:
# Check for any missing values
train.isnull().sum().any(), test.isnull().sum().any()

In [ ]:
def check_missing_values(df):
    # Get count of missing values
    total = df.isnull().sum()
    # Get percentage of missing values
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'percent_missing': percent_missing,
                                    'total': total,
                                    'type': df.dtypes})
    # Sort by pct
    missing_value_df.sort_values('percent_missing', ascending=False, inplace=True)
    
    return missing_value_df

In [ ]:
# Check for missing value pct in train
check_missing_values(train)

In [ ]:
# Check for missing value pct in test
check_missing_values(test)

In [ ]:
# Check for duplicates
train.duplicated().any(), test.duplicated().any()

## Data Cleaning

In [ ]:
train.shape, test.shape

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
sns.histplot(data=train, x="pm2_5", color="blue")

In [ ]:
# plot distribution after log transformation
sns.histplot(np.log(train['pm2_5']), color="blue")

In [ ]:
from sklearn.preprocessing import PowerTransformer

transformer = PowerTransformer(method="box-cox", standardize=False)
sns.histplot(transformer.fit_transform(train['pm2_5'].values.reshape(-1, 1)), color="blue")

In [ ]:
def get_data_collection_report(df, kind="train"):
    df["date"] = pd.to_datetime(df["date"])
    
    start_date = df["date"].min()
    end_date = df["date"].max()
    duration = end_date - start_date
    
    print(f"Data in {kind} covers the period {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print(f"Length of period: {duration.days + 1} days")

In [ ]:
# print a report of collection dates for train set
get_data_collection_report(train)

In [ ]:
# print a report of collection dates for test set
get_data_collection_report(test, kind="test")

In [ ]:
# # drop pm2_5 values above 300
# train = train[(train.pm2_5 <= 200) & (train.pm2_5 >= 10)]
# train = train.reset_index().drop(columns=["index"])

### Handle Missing Values

In [ ]:
# !git clone https://github.com/flaviagiammarino/brits-tensorflow.git

In [ ]:
# sys.path.append("brits-tensorflow")

# from brits_tensorflow.model import BRITS


# all_df = pd.concat([train, test])
# all_df = all_df.set_index("ID")

# all_df['date'] = pd.to_datetime(all_df['date'])
# all_df = all_df.sort_values(by=["site_latitude","date"])

# missing = check_missing_values(train)
# cols_with_missing = missing[(missing.percent_missing > 0)].index

# num_steps = 1
# # Fit the model
# model = BRITS(
#     x=all_df[cols_with_missing].to_numpy(),
#     units=100,
#     timesteps=num_steps)

# model.fit(
#     learning_rate=1e-4,
#     batch_size=16,
#     epochs=50,
#     verbose=True)
    
# # Impute the missing values
# imputations = model.predict(x=all_df[cols_with_missing].to_numpy())

# # stations = list()

# # for i, lat in enumerate(all_df["site_latitude"].unique()):
# #     site_data = all_df[all_df.site_latitude == lat]
    
# #     # Fit the model
# #     model = BRITS(
# #         x=site_data[cols_with_missing].to_numpy(),
# #         units=100,
# #         timesteps=200)
    
# #     model.fit(
# #         learning_rate=0.001,
# #         batch_size=16,
# #         epochs=200,
# #         verbose=False
# #     )
    
# #     # Impute the missing values
# #     imputations = model.predict(x=site_data[cols_with_missing].to_numpy())
    
# #     site_data[cols_with_missing] = imputations.values
# #     stations.append(site_data)
    
# #     print(f'Finished imputing for station at {lat} ({i+1}/{all_df["site_latitude"].nunique()})')

In [ ]:
# all_df[cols_with_missing] = imputations.values

In [ ]:
# train = all_df[~all_df.pm2_5.isna()]
# test = all_df[all_df.pm2_5.isna()]

In [ ]:
# train.to_csv("train_imputed.csv")
# test.to_csv("test_imputed.csv")

In [ ]:
# train = pd.read_csv("../input/layerai-air-quality-prediction-challenge/train_imputed.csv")
# test = pd.read_csv("../input/layerai-air-quality-prediction-challenge/test_imputed.csv")

## Feature Engineering

In [ ]:
train["date"] = pd.to_datetime(train["date"])
test["date"] = pd.to_datetime(test["date"])

In [ ]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period *2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1 # periodoc and include bias = True
    return SplineTransformer(degree=degree,
                            n_knots=n_knots,
                            knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
                            extrapolation="periodic",
                            include_bias=True)

def extract_time_features(df):

    # Extract raw ordinal time-related features
    df["year"] = df.date.dt.year
    df["month"] = df.date.dt.month
    df["day"] = df.date.dt.day
    df["weekday"] = df.date.dt.dayofweek
    df["day_of_year"] = df.date.dt.day_of_year
    df["week_of_year"] = df.date.dt.weekofyear
    df["quarter"] = df.date.dt.quarter
    df["is_month_start"] = df.date.dt.is_month_start
    df["is_month_end"] = df.date.dt.is_month_end
    df["is_quarter_start"] = df.date.dt.is_quarter_start
    df["is_quarter_end"] = df.date.dt.is_quarter_end



    dayname = df.date.dt.day_name()
    df["is_weekend"] = np.where(dayname.isin(["Saturday", "Sunday"]), "Yes", "No")

    # Extract trigonometric features
    df["month_sin"] = sin_transformer(12).fit_transform(df["month"])
    df["month_cos"] = cos_transformer(12).fit_transform(df["month"])

    df["weekday_sin"] = sin_transformer(7).fit_transform(df["weekday"])
    df["weekday_cos"] = cos_transformer(7).fit_transform(df["weekday"])

    # Extract periodic spline-based features
    spline_cols_month = [f"cyclic_month_spline_{i}" for i in range(1, 7)]
    spline_cols_weekday = [f"cyclic_weekday_spline_{i}" for i in range(1, 4)]

    df[spline_cols_month] = periodic_spline_transformer(12, n_splines=6).fit_transform(np.array(df.month).reshape(-1,1))
    df[spline_cols_weekday] = periodic_spline_transformer(7, n_splines=3).fit_transform(np.array(df.weekday).reshape(-1,1))


    return df

In [ ]:
# Extract date related features
# Questions to ask: Is there any relationship between date and PM2.5 concentration?
# Are there some trends based on date, maybe seasonality?
train = extract_time_features(train)
test = extract_time_features(test)

In [ ]:
# Extract geospatial features
def extract_geo_spatial_feat(df):
    # Add polar coordinates
    df["polar_radius"] = np.sqrt(np.square(df["site_longitude"]) + np.square(df["site_latitude"]))
    df["phi"] = np.arctan2(df["site_latitude"], df["site_longitude"])
    
    # Add rotational cartesian coordinates
    df["rot_30_x"] = np.cos(30 * np.pi / 180) * df["site_longitude"] + np.sin(30 * np.pi / 180) * df["site_latitude"]
    df["rot_30_y"] = np.sin(30 * np.pi / 180) * df["site_longitude"] - np.cos(30 * np.pi /180) * df["site_latitude"]
    
    df["rot_45_x"] = np.cos(45 * np.pi / 180) * df["site_longitude"] + np.sin(45 * np.pi / 180) * df["site_latitude"]
    df["rot_45_y"] = np.sin(45 * np.pi / 180) * df["site_longitude"] - np.cos(45 * np.pi /180) * df["site_latitude"]
    
    df["rot_60_x"] = np.cos(60 * np.pi / 180) * df["site_longitude"] + np.sin(60 * np.pi / 180) * df["site_latitude"]
    df["rot_60_y"] = np.sin(60 * np.pi / 180) * df["site_longitude"] - np.cos(60 * np.pi /180) * df["site_latitude"]
    
    return df

In [ ]:
train = extract_geo_spatial_feat(train)
test = extract_geo_spatial_feat(test)

In [ ]:
# Transform latitude and longitude coordinates using PCA
coords = np.vstack((train[["site_latitude", "site_longitude"]],
                   test[["site_latitude", "site_longitude"]]))

pca = PCA()
pca.fit(coords)

train["pca_0"] = pca.transform(train[["site_latitude", "site_longitude"]])[:,0]
train["pca_1"] = pca.transform(train[["site_latitude", "site_longitude"]])[:,1]

test["pca_0"] = pca.transform(test[["site_latitude", "site_longitude"]])[:,0]
test["pca_1"] = pca.transform(test[["site_latitude", "site_longitude"]])[:,1]

In [ ]:
feature_counts = pd.concat([train, test]).groupby("is_weekend").size()
train["weekend_Counts"] = train["is_weekend"].apply(lambda x: feature_counts[x])
test["weekend_Counts"] = test["is_weekend"].apply(lambda x: feature_counts[x])

In [ ]:
# Binning temperature and humidity
est = KBinsDiscretizer(n_bins=5, encode='onehot-dense', strategy='uniform')
to_discretize = ["temp_mean", "humidity"]
bin_cols = [f"temp_mean_bin{bin}" for bin in range(1, 6)]
bin_cols_humidity = [f"humidity_bin{bin}" for bin in range(1, 6)]
bin_cols.extend(bin_cols_humidity)

train[bin_cols] = est.fit_transform(train[to_discretize].fillna(train[to_discretize].mean()))
test[bin_cols] = est.transform(test[to_discretize].fillna(test[to_discretize].mean()))

In [ ]:
train["is_train"] = 1

In [ ]:
all_df = pd.concat([train, test])
all_df = all_df.sort_values(by=["site_latitude", "date"])

In [ ]:
def create_time_series_features(df, lat):
    df = df[df.site_latitude == lat]
    df["humidity_lag1"] = df["humidity"].shift(1)
    df["humidity_lag2"] = df["humidity"].shift(2)
    df["humidity_lag3"] = df["humidity"].shift(3)
    df["humidity_lag4"] = df["humidity"].shift(4)
    df["humidity_lag5"] = df["humidity"].shift(5)
    df["humidity_lag6"] = df["humidity"].shift(6)
    df["humidity_lag7"] = df["humidity"].shift(7)
    
    df["humidity_lagback1"] = df["humidity"].shift(-1)
    df["humidity_lagback2"] = df["humidity"].shift(-2)
    df["humidity_lagback3"] = df["humidity"].shift(-3)
    df["humidity_lagback4"] = df["humidity"].shift(-4)
    df["humidity_lagback5"] = df["humidity"].shift(-5)
    df["humidity_lagback6"] = df["humidity"].shift(-6)
    df["humidity_lagback7"] = df["humidity"].shift(-7)
    
    df["temp_lag1"] = df["temp_mean"].shift(1)
    df["temp_lag2"] = df["temp_mean"].shift(2)
    df["temp_lag3"] = df["temp_mean"].shift(3)
    df["temp_lag4"] = df["temp_mean"].shift(4)
    df["temp_lag5"] = df["temp_mean"].shift(5)
    df["temp_lag6"] = df["temp_mean"].shift(6)
    df["temp_lag7"] = df["temp_mean"].shift(7)
    
    df["temp_lagback1"] = df["temp_mean"].shift(-1)
    df["temp_lagback2"] = df["temp_mean"].shift(-2)
    df["temp_lagback3"] = df["temp_mean"].shift(-3)
    df["temp_lagback4"] = df["temp_mean"].shift(-4)
    df["temp_lagback5"] = df["temp_mean"].shift(-5)
    df["temp_lagback6"] = df["temp_mean"].shift(-6)
    df["temp_lagback7"] = df["temp_mean"].shift(-7)
    
    
    df["humidity_rolling_mean"] = df["humidity"].rolling(window=3).mean()
    df["humidity_rolling_max"] = df["humidity"].rolling(window=3).max()
    df["humidity_rolling_min"] = df["humidity"].rolling(window=3).min()
    df["humidity_rolling_sum"] = df["humidity"].rolling(window=3).sum()
    df["humidity_rolling_std"] = df["humidity"].rolling(window=3).std()

    df["temp_rolling_mean"] = df["temp_mean"].rolling(window=3).mean()
    df["temp_rolling_max"] = df["temp_mean"].rolling(window=3).max()
    df["temp_rolling_min"] = df["temp_mean"].rolling(window=3).min()
    df["temp_rolling_sum"] = df["temp_mean"].rolling(window=3).sum()
    df["temp_rolling_std"] = df["temp_mean"].rolling(window=3).std()
    
    df["humidity_expanding_mean"] = df["humidity"].expanding(min_periods=2).mean()
    df["humidity_expanding_sum"] = df["humidity"].expanding(min_periods=2).sum()
    df["humidity_expanding_max"] = df["humidity"].expanding(min_periods=2).max()
    df["humidity_expanding_min"] = df["humidity"].expanding(min_periods=2).min()
    df["humidity_expanding_std"] = df["humidity"].expanding(min_periods=2).std()

    df["temp_expanding_mean"] = df["temp_mean"].expanding(min_periods=2).mean()
    df["temp_expanding_sum"] = df["temp_mean"].expanding(min_periods=2).sum()
    df["temp_expanding_max"] = df["temp_mean"].expanding(min_periods=2).max()
    df["temp_expanding_min"] = df["temp_mean"].expanding(min_periods=2).min()
    df["temp_expanding_std"] = df["temp_mean"].expanding(min_periods=2).std()
    
    df = df.fillna(0)
    
    return df

In [ ]:
def calc_rec_dist(lat1, lon1, lat2, lon2):
    """
    Calculates the rectilinear distance between two pointss
    :param lat1: latitude of first point
    :param lon1: longitude of first point
    :param lat2: latitude of second point
    :param lon2: longitude of second point
    :return: distance
    """
    r = np.sqrt(np.square(lat1 -lat2) + np.square(lon1 - lon2))
    return r

# Calculate distance between the unique location in test and the locations in train
distances = []
for lat, lon in zip(all_df["site_latitude"].unique(), all_df["site_longitude"].unique()):
    dist = calc_rec_dist(lat, lon, 0.332609275, 32.610047)
    distances.append([dist, lat])

# get the 3 closest locations
distances.sort(key=lambda x: x[0])
nearby_lats = np.array(distances[1:4])[:,1]

# Let's see how many data points there are at this site
unique_location_ids = all_df[all_df.site_latitude == 0.332609275].index
len(unique_location_ids)

In [ ]:
temp_df = pd.DataFrame()
sites = all_df["site_latitude"].unique()
for lat in tqdm(sites, total=len(sites)):
    if lat in unique_location_ids:
        # Calculate the time series features for unique location
        # by simulating it being in each of the three cloest sites, then
        # averaging the result. You can also try weighted distance
        # averaging since the statistics are likely to be most similar
        # with the cloest site

        group = all_df[all_df.site_latitude == lat]
        group1 = create_time_series_features(all_df,nearby_lats[0])
        group2 = create_time_series_features(all_df,nearby_lats[1])
        group3 = create_time_series_features(all_df,nearby_lats[2])
        
        cols = group1.columns[-34:]
        group[cols] = (group1[cols] + group2[cols] + group3[cols])/3
    else:
        group = create_time_series_features(all_df, lat)
    
    temp_df =pd.concat([temp_df, group])

In [ ]:
all_df = temp_df.copy()

In [ ]:
all_df["humidity_lagback_diff1"] =  all_df["humidity"] - all_df["humidity_lagback1"]
all_df["humidity_lagback_diff2"] =  all_df["humidity"] - all_df["humidity_lagback2"]
all_df["humidity_lagback_diff3"] =  all_df["humidity"] - all_df["humidity_lagback3"]
all_df["humidity_lagback_diff4"] =  all_df["humidity"] - all_df["humidity_lagback4"]
all_df["humidity_lagback_diff5"] =  all_df["humidity"] - all_df["humidity_lagback5"]
all_df["humidity_lagback_diff6"] =  all_df["humidity"] - all_df["humidity_lagback6"]
all_df["humidity_lagback_diff7"] =  all_df["humidity"] - all_df["humidity_lagback7"]

all_df["temp_lagback_diff1"] = all_df["temp_mean"] - all_df["temp_lagback1"]
all_df["temp_lagback_diff2"] = all_df["temp_mean"] - all_df["temp_lagback2"]
all_df["temp_lagback_diff3"] = all_df["temp_mean"] - all_df["temp_lagback3"]
all_df["temp_lagback_diff4"] = all_df["temp_mean"] - all_df["temp_lagback4"]
all_df["temp_lagback_diff5"] = all_df["temp_mean"] - all_df["temp_lagback5"]
all_df["temp_lagback_diff6"] = all_df["temp_mean"] - all_df["temp_lagback6"]
all_df["temp_lagback_diff7"] = all_df["temp_mean"] - all_df["temp_lagback7"]

In [ ]:
train = all_df[all_df.is_train == 1]
test = all_df[all_df.is_train != 1]

In [ ]:
train.drop(columns=["is_train"], inplace=True)
test.drop(columns=["is_train"], inplace=True)

In [ ]:
# Sanity check
len(test)

## Pre-processing

In [ ]:
# Target Encoding
class TargetEncode(BaseEstimator, TransformerMixin):

    def __init__(self, categories='auto', k=1, f=1, noise_level=0, random_state=None):
        if type(categories)==str and categories!='auto':
            self.categories = [categories]
        else:
            self.categories = categories
        self.k = k
        self.f = f
        self.noise_level = noise_level
        self.encodings = dict()
        self.prior = None
        self.random_state = random_state
        
    def add_noise(self, series, noise_level):
        return series * (1 + noise_level * np.random.randn(len(series)))
    
    def fit(self, X, y=None):
        if type(self.categories)=='auto':
            self.categories = np.where(X.dtypes == type(object()))[0]
            
        temp = X.loc[:, self.categories].copy()
        temp['Label'] = y
        self.prior = np.mean(y)
        for variable in self.categories:
            avg = (temp.groupby(by=variable)['Label']
                   .agg(['mean', 'count']))
            # Compute smoothing
            smoothing = (1 / (1 + np.exp(-(avg['count'] - self.k) /
                                         self.f)))
            # The bigger the count the less full_avg is accounted
            self.encodings[variable] = dict(self.prior * (1 -
                                                          smoothing) + avg['mean'] * smoothing)
        return self

    def transform(self, X):
        Xt = X.copy()
        for variable in self.categories:
            Xt[variable].replace(self.encodings[variable], inplace=True)
            unknown_value = {value:self.prior for value in 
                             X[variable].unique() 
                             if value not in
                             self.encodings[variable].keys()}
            if len(unknown_value) > 0:
                Xt[variable].replace(unknown_value, inplace=True)
            Xt[variable] = Xt[variable].astype(float)
            if self.noise_level > 0:
                if self.random_state is not None:
                    np.random.seed(self.random_state)
                Xt[variable] = self.add_noise(Xt[variable], self.noise_level)
        return Xt

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)

In [ ]:
# First duplication this column, we'll need it later
train["station"] = train["device"]
test["station"] = test["device"]

In [ ]:
# Target encode device column
te = TargetEncode(categories=["device"])
te.fit(train.drop(columns=["pm2_5"]), train['pm2_5'])
train = te.transform(train)
test = te.transform(test)

In [ ]:
nearby_lats

In [ ]:
# for the unique location let's find the average for 3 nearby locations
loc1 = train[train.site_latitude == nearby_lats[0]]["device"]
loc2 = train[train.site_latitude == nearby_lats[1]]["device"]
loc3 = train[train.site_latitude == nearby_lats[2]]["device"]

avg = (loc1.iloc[0] + loc2.iloc[0] + loc3.iloc[0]) / 3

test = test.reset_index().drop(columns=["index"])
idx = test[test.site_latitude == 0.332609275].index
test["device"].iloc[idx] = avg

In [ ]:
train = train.set_index("ID")
test = test.set_index("ID")

## Modeling

In [ ]:
STATIC_REAL = ["station", "device", "pca_0","pca_1", "site_latitude", "site_longitude", "polar_radius", "phi", "rot_30_x", 
               "rot_30_y", "rot_45_x", "rot_45_y", "rot_60_x","rot_60_y", 'weekend_Counts',
              'month_sin', 'month_cos', 'weekday_sin', 'weekday_cos', 'cyclic_month_spline_1', 'cyclic_month_spline_2',
               'cyclic_month_spline_3', 'cyclic_month_spline_4', 'cyclic_month_spline_5',
              'cyclic_month_spline_6', 'cyclic_weekday_spline_1', 'cyclic_weekday_spline_2']

DYNAMIC_CAT = ['year', 'month', 'day', 'weekday', 'quarter', 'is_month_start', 'is_month_end', 'is_quarter_start', 
               'is_quarter_end', 'is_weekend']

DYNAMIC_REAL = ["time_idx", "humidity", "temp_mean", 'SulphurDioxide_SO2_column_number_density',
                'SulphurDioxide_SO2_column_number_density_amf', 'SulphurDioxide_SO2_slant_column_number_density', 
                'SulphurDioxide_cloud_fraction', 'SulphurDioxide_sensor_azimuth_angle', 'SulphurDioxide_sensor_zenith_angle',
                'SulphurDioxide_solar_azimuth_angle', 'SulphurDioxide_solar_zenith_angle', 'SulphurDioxide_SO2_column_number_density_15km',
                'CarbonMonoxide_CO_column_number_density', 'CarbonMonoxide_H2O_column_number_density', 'CarbonMonoxide_cloud_height',
                'CarbonMonoxide_sensor_altitude', 'CarbonMonoxide_sensor_azimuth_angle', 'CarbonMonoxide_sensor_zenith_angle',
                'CarbonMonoxide_solar_azimuth_angle', 'CarbonMonoxide_solar_zenith_angle', 'NitrogenDioxide_NO2_column_number_density',
                'NitrogenDioxide_tropospheric_NO2_column_number_density', 'NitrogenDioxide_stratospheric_NO2_column_number_density',
                'NitrogenDioxide_NO2_slant_column_number_density', 'NitrogenDioxide_tropopause_pressure', 'NitrogenDioxide_absorbing_aerosol_index',
                'NitrogenDioxide_cloud_fraction', 'NitrogenDioxide_sensor_altitude', 'NitrogenDioxide_sensor_azimuth_angle',
                'NitrogenDioxide_sensor_zenith_angle', 'NitrogenDioxide_solar_azimuth_angle', 'NitrogenDioxide_solar_zenith_angle',
                'Formaldehyde_tropospheric_HCHO_column_number_density', 'Formaldehyde_tropospheric_HCHO_column_number_density_amf',
                'Formaldehyde_HCHO_slant_column_number_density', 'Formaldehyde_cloud_fraction', 'Formaldehyde_solar_zenith_angle',
                'Formaldehyde_solar_azimuth_angle', 'Formaldehyde_sensor_zenith_angle', 'Formaldehyde_sensor_azimuth_angle',
                'UvAerosolIndex_absorbing_aerosol_index', 'UvAerosolIndex_sensor_altitude', 'UvAerosolIndex_sensor_azimuth_angle', 
                'UvAerosolIndex_sensor_zenith_angle', 'UvAerosolIndex_solar_azimuth_angle', 'UvAerosolIndex_solar_zenith_angle',
                 'Ozone_O3_column_number_density', 'Ozone_O3_column_number_density_amf', 'Ozone_O3_slant_column_number_density',
                 'Ozone_O3_effective_temperature', 'Ozone_cloud_fraction', 'Ozone_sensor_azimuth_angle', 'Ozone_sensor_zenith_angle',
                 'Ozone_solar_azimuth_angle', 'Ozone_solar_zenith_angle', 'Cloud_cloud_fraction', 'Cloud_cloud_top_pressure', 'Cloud_cloud_top_height',
                 'Cloud_cloud_base_pressure', 'Cloud_cloud_base_height', 'Cloud_cloud_optical_depth', 'Cloud_surface_albedo', 'Cloud_sensor_azimuth_angle',
                 'Cloud_sensor_zenith_angle', 'Cloud_solar_azimuth_angle', 'Cloud_solar_zenith_angle']

DYNAMIC_REAL_UNKNOWN = ["pm2_5", 'temp_mean_bin1', 'temp_mean_bin2', 'temp_mean_bin3', 'temp_mean_bin4', 'temp_mean_bin5',
                         'humidity_bin1', 'humidity_bin2', 'humidity_bin3', 'humidity_bin4', 'humidity_bin5', 'humidity_lag1',
                        'humidity_lag2', 'humidity_lag3', 'humidity_lag4', 'humidity_lag5', 'humidity_lag6', 'humidity_lag7',
                        'humidity_lagback1', 'humidity_lagback2', 'humidity_lagback3', 'humidity_lagback4', 'humidity_lagback5',
                        'humidity_lagback6', 'humidity_lagback7', 'temp_lag1', 'temp_lag2', 'temp_lag3', 'temp_lag4', 'temp_lag5',
                        'temp_lag6', 'temp_lag7', 'temp_lagback1', 'temp_lagback2', 'temp_lagback3', 'temp_lagback4',
                        'temp_lagback5', 'temp_lagback6', 'temp_lagback7', 'humidity_rolling_mean', 'humidity_rolling_max',
                        'humidity_rolling_min', 'humidity_rolling_sum', 'humidity_rolling_std', 'temp_rolling_mean', 'temp_rolling_max', 
                        'temp_rolling_min', 'temp_rolling_sum', 'temp_rolling_std','humidity_expanding_mean', 'humidity_expanding_sum',
                        'humidity_expanding_max', 'humidity_expanding_min', 'humidity_expanding_std', 'temp_expanding_mean',
                        'temp_expanding_sum','temp_expanding_max', 'temp_expanding_min','temp_expanding_std', 'humidity_lagback_diff1', 
                        'humidity_lagback_diff2','humidity_lagback_diff3','humidity_lagback_diff4','humidity_lagback_diff5',
                        'humidity_lagback_diff6','humidity_lagback_diff7','temp_lagback_diff1','temp_lagback_diff2','temp_lagback_diff3',
                        'temp_lagback_diff4','temp_lagback_diff5','temp_lagback_diff6','temp_lagback_diff7']

In [ ]:
class DataPipeline():
    """
    This class contains utilities for creating a time_index, adding
    # pseudo labels, and transforming datatypes and reducing memory usage
    objects.
    """
    
    def __init__(self, df_train: pd.DataFrame, df_test: pd.DataFrame, add_pseudo_labels=False):
#         self.cut_off = valid_cutoff
        self.train = df_train
        self.test = df_test
        self.pseudo_label = add_pseudo_labels
        
        
    def transform(self):
        df = pd.concat([self.train, self.test])
        
        # Add time index
        df['time_idx'] = (df['date'].dt.date - df['date'].dt.date.min()).dt.days
        self.train["time_idx"] = df["time_idx"].iloc[:len(self.train)]
        self.test["time_idx"] = df["time_idx"].iloc[len(self.train):]
        
        # Convert categorical column to numerical form to use it as a group id
        le = LabelEncoder()
        le.fit(df["station"])
        
        self.train["station"] = le.transform(self.train["station"])
        self.test["station"] = le.transform(self.test["station"])

        if self.pseudo_label:
            # Add some pseudo labels
            pseudo_samples = self.test[self.test["date"] <= "2020-09-20"]
            pseudo_labels = layer.get_dataset("nkosana_mlandu/zindi-air-quality-prediction-challenge/datasets/pseudo_labels:1.1").to_pandas().values
            pseudo_samples["pm2_5"] = pseudo_labels

            # Filter pseudo samples to remove rare stations
            # Station 34 and 35 are not in the train set
            pseudo_samples = pseudo_samples[pseudo_samples["station"] < 34]
        
        self.train = pd.concat([self.train, pseudo_samples])
        
        # Pytorch expects all categorical columns to be "str"
        self.train[DYNAMIC_CAT] = self.train[DYNAMIC_CAT].astype("str")
        self.test[DYNAMIC_CAT] = self.test[DYNAMIC_CAT].astype("str")

        self.train = reduce_mem_usage(self.train)
        self.test = reduce_mem_usage(self.test)
        return self.train, self.test


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64',
                'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:

# A Note on pseudo labeling: We only added up to 1 month into the test set per 
# each time series, and all pseudo labelled examples fall into
# range > max_prediction length thus will be added to the validation set.
# There's no risk of contaminating the train set, but however 
# on the downside this impacts the evaluation metric
pipe = DataPipeline(train, test, add_pseudo_labels=True)
train, test = pipe.transform()

In [ ]:
train["time_idx"].max()

In [ ]:
max_prediction_length = 134 # predict full length of test set
max_encoder_length = train["date"].nunique()

training = TimeSeriesDataSet(
    train,
    time_idx="time_idx",
    target="pm2_5",
    group_ids=["station"],
    min_encoder_length=0,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_reals=STATIC_REAL,
    time_varying_known_reals=DYNAMIC_REAL,
    time_varying_known_categoricals=DYNAMIC_CAT,
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=DYNAMIC_REAL_UNKNOWN,
    categorical_encoders={
       'month': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
        'day': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True)   
    },
#     target_normalizer=GroupNormalizer(
#         groups=["station"], transformation="softplus"
#     ),  # use softplus and normalize by group
    target_normalizer=GroupNormalizer(
        groups=["station"], transformation="log"
    ), # use log transfooooration and normalize by group
    scalers={"StandardScaler": StandardScaler()},
    add_relative_time_idx=False,
    add_target_scales=False,
    add_encoder_length=False,
    allow_missing_timesteps=True,
)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, train, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [ ]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

In [ ]:
actuals.shape

In [ ]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(     
    accelerator="gpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=1,  # 7 quantiles by default
    loss=SMAPE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

### Tuning Hyperparameters

In [ ]:
# SKIP
#study = optimize_hyperparameters(
#    train_dataloader,
#    val_dataloader,
#    model_path="optuna_test",
#    n_trials=200, 
#    max_epochs=50,
#    gradient_clip_val_range=(0.01, 1.0),
#    hidden_size_range=(8, 128),
#    hidden_continuous_size_range=(8, 128),
#    attention_head_size_range=(1, 4),
#    learning_rate_range=(0.001, 0.1),
#    dropout_range=(0.1, 0.3),
#    trainer_kwargs=dict(limit_train_batches=30),
#    reduce_on_plateau_patience=4,
#    use_learning_rate_finder=False,
#)
#with open("test_study.pkl", "wb") as fout:
#    pickle.dump(study, fout)

In [ ]:
#  find optimal learning rate
res = trainer.tuner.lr_find(
     tft,
     train_dataloaders=train_dataloader,
     val_dataloaders=val_dataloader,
     max_lr=10,
     min_lr=1e-9)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

### Training

In [ ]:
MAX_EPOCHS = 30
GRADIENT_CLIP_VAL = 0.1
LEARNING_RATE = 0.6
HIDDEN_SIZE = 16
ATTENTION_HEAD_SIZE = 1
DROPOUT = 0.2
HIDDEN_CONTINOUS_SIZE = 8
OUTPUT_SIZE = 1
LOSS = SMAPE()

# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard
#logger = LayerLogger(project_name='zindi-air-quality-prediction-challenge', api_key='[API-KEY]')

    
trainer = pl.Trainer(
        max_epochs=MAX_EPOCHS,
        accelerator="gpu",
        enable_model_summary=True,
        gradient_clip_val=GRADIENT_CLIP_VAL,         
        #limit_train_batches=30, # comment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback],
        logger=logger)

@model("TFT-beta")
def train_model():
    

    model = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=LEARNING_RATE,
#         lstm_layers=LSTM_LAYERS,
        hidden_size=HIDDEN_SIZE,  
        attention_head_size=ATTENTION_HEAD_SIZE,
        dropout=DROPOUT,          
        hidden_continuous_size=HIDDEN_CONTINOUS_SIZE,  
        output_size=OUTPUT_SIZE,
        loss=LOSS,
        reduce_on_plateau_patience=4,
    )
    
    params = {"learning_rate": LEARNING_RATE,
#               "lstm_layers": LSTM_LAYERS,
              "hidden_size": HIDDEN_SIZE,  
              "attention_head_size": ATTENTION_HEAD_SIZE,
              "dropout": DROPOUT,          
              "hidden_continuous_size": HIDDEN_CONTINOUS_SIZE,  
              "output_size": OUTPUT_SIZE,
              "loss": 'SMAPE'}
    layer.log(params)  # Log parameters

    print(f"Number of parameters in network: {model.size()/1e3:.1f}k")     
    layer.log({"Number of parameters in network": f"{model.size()/1e3:.1f}k"})
    
    # fit network
    trainer.fit(
        model,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader)
    
    
    # load the best model according to the validation loss
    # (given that we use early stopping, this is not necessarily the last epoch)
    best_model_path = trainer.checkpoint_callback.best_model_path
    best_model = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    
    return best_model

In [ ]:
# NOTE: Each epoch takes around 20 mins on the complete dataset.
# Thus running up to MAX_EPOCHS will take approx. 10 hours

In [ ]:
best_model = train_model()
# layer.run([train_model], debug=True)

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir lightning_logs

### Model Evaluation

In [ ]:
@model("TFT-evaluation")
def evaluate_model():
    # load the best model according to the validation loss
    # (given that we use early stopping, this is not necessarily the last epoch)
    best_model_path = trainer.checkpoint_callback.best_model_path
    best_model = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    
    # calcualte mean absolute error on validation set
    actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
    predictions = best_model.predict(val_dataloader)
    valid_mae = (actuals - predictions).abs().mean()
    
    print(f"Valid MAE: {valid_mae}")
    layer.log({"Validation MAE": f"{valid_mae}"})
    
    sm = SMAPE()
    print(f"Validation median SMAPE loss: {sm.loss(actuals, predictions).mean(axis = 1).median().item()}")
    layer.log({"Validation median SMAPE loss": f"{sm.loss(actuals, predictions).mean(axis = 1).median().item()}"})
    
    
    # raw predictions are a dictionary from which all kind of information including quantiles can be extracted
    raw_predictions, x = best_model.predict(val_dataloader, mode="raw", return_x=True)
    for idx in range(34):  # plot 34 examples
        fig = best_model.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True)
        layer.log({"Plot of predictions":fig}, step=idx)
    
    return best_model

In [ ]:
best_model = evaluate_model()
# layer.run([evaluate_model])

### Error analysis

In [ ]:
@model("TFT-error-_analysis")
def model_error_analysis():
    # raw predictions are a dictionary from which all kind of information including quantiles can be extracted
    raw_predictions, x = best_model.predict(val_dataloader, mode="raw", return_x=True)

    # calcualte metric by which to display
    predictions = best_model.predict(val_dataloader)
    mean_losses = SMAPE(reduction="none")(predictions, actuals).mean(1)
    indices = mean_losses.argsort(descending=True)  # sort losses
    
    for idx in range(10):  # plot 10 examples
         fig = best_model.plot_prediction(
            x, raw_predictions, idx=indices[idx], add_loss_to_title=SMAPE(quantiles=best_model.loss.quantiles))
         layer.log({f"Worst performers": fig}, step=idx)
    predictions, x = best_model.predict(val_dataloader, return_x=True)        
    predictions_vs_actuals = best_model.calculate_prediction_actual_by_variable(x, predictions)
    fig_2 = best_model.plot_prediction_actual_by_variable(predictions_vs_actuals);
    layer.log({"Predicted vs Actual": fig_2})
    
    return best_model

In [ ]:
_ = model_error_analysis()

### Inference

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# unique devices = aq_91 and aq_98, latitude = 0.332609275
# Ideas: replace device with device from 3 nearest locations, then average the predictions. Also try weighted averaging based on distance to location.

In [ ]:
# Replace aq_91 and aq_98 with APYZC5J7, aq_36, aq_74

In [ ]:
#@dataset("generate_predictions")
def generate_predictions(df, plot=False):
    # select last 557 days from data (max_encoder_length is 557)
    encoder_data = train[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]
    #cutoff = "2020-2-20"
    #end_train = "2020-08-20"
    #encoder_data = train[lambda x: (x.date >= cutoff) & (x.date < end_train)]    
    
    decoder_data = df.copy()
    
    # combine encoder and decoder data
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    
    predictions = best_model.predict(new_prediction_data, return_x=False)
    
    if plot:
        new_raw_predictions, new_x = best_model.predict(new_prediction_data, mode="raw", return_x=True)
        for idx in range(34):  # plot 34 examples
            fig = best_model.plot_prediction(new_x, new_raw_predictions, idx=idx, show_future_observed=False)
            
            #layer.log({"Inference": fig}, step=idx)
        
        interpretation = best_model.interpret_output(new_raw_predictions, reduction="sum")
        fig_2 = best_model.plot_interpretation(interpretation)
        #layer.log({"Model intepretation": fig_2})
    

    predictions_df = pd.DataFrame(predictions.numpy()).T
    predictions_df['time_idx'] = sorted(df['time_idx'].unique())

    predictions_df2 = pd.melt(predictions_df, id_vars=['time_idx'])
    predictions_df2.rename(columns = {'value':'pm2_5', 'variable':'station'}, inplace = True)

    #now copying the results back to df_test as per location and time
    result = pd.merge(df, predictions_df2, on=["time_idx", "station"])
    result.rename(columns={'pm2_5_y': "pm2_5"}, inplace=True)
    result["ID"] = df.index
    
    return result

In [ ]:
def generate_predictions_unique_stations(strategy="mean"):
    test_pseudo_1 = test.copy()
    test_pseudo_1["station"] = test_pseudo_1["station"].replace({34:15, 35: 15})
    pred_1 = generate_predictions(test_pseudo_1)
    pred_1 = pred_1[pred_1.site_latitude == 0.332609275]
    
    test_pseudo_2 = test.copy()
    test_pseudo_2["station"] = test_pseudo_2["station"].replace({34:31, 35: 31})
    pred_2 = generate_predictions(test_pseudo_2)
    pred_2 = pred_2[pred_2.site_latitude == 0.332609275]
    
    test_pseudo_3 = test.copy()
    test_pseudo_3["station"] = test_pseudo_3["station"].replace({34:6, 35: 6})
    pred_3 = generate_predictions(test_pseudo_3)
    pred_3 = pred_3[pred_3.site_latitude == 0.332609275]
    
    
    if strategy =="mean":
        preds = np.mean([pred_1["pm2_5"].values, pred_2["pm2_5"].values, pred_3["pm2_5"]], axis=0)
    elif strategy == "median":
        preds = np.median([pred_1["pm2_5"].values, pred_2["pm2_5"].values, pred_3["pm2_5"]], axis=0)

    elif strategy == "mean_weighted":
        distances_to_station = np.array(distances[1:4])[:,0]    
        weights = sorted([dist / np.sum(distances_to_station) for dist in distances_to_station], 
                        reverse=True) 
    
        preds = np.average([pred_1["pm2_5"].values, pred_2["pm2_5"].values, pred_3["pm2_5"]], 
                             weights=weights, axis=0)
        
        
    df = test[test.site_latitude == 0.332609275]
    df["pm2_5"] = preds
    
    return df

In [ ]:
# test['time_idx'] = (test['date'].dt.date - train['date'].dt.date.min()).dt.days
test_clean = test[~test.station.isin([34, 35])]

known_stations = generate_predictions(test_clean,plot=False)
new_stations = generate_predictions_unique_stations(strategy="mean_weighted")
new_stations = new_stations.reset_index()

test_preds = pd.concat([known_stations, new_stations])

In [ ]:
test_preds[["ID", "station", "site_latitude", "site_longitude", "pm2_5"]]

### Submission

In [ ]:
test_preds[["ID", "pm2_5"]].to_csv("best_submission.csv", index=False)

## Acknowledgements and concluding remarks
We acknowledge the competition organisers [Layer](https://layer.ai), the dataset providers [AirQo](https://platform.airqo.net) and to our beloved friends at [Zindi](https://zindi.africa) thank you for coninuing to advance the cause of Data Science in Africa. To our fellow data scientists we hope you had fun, and most importantly you learnt a lot through the competition and that you'll continue advancing your skills. Let's keep building data solutions for Africa, Zindisha!